# Notebook 05: Computer Vision - Object Detection

**Learning Objectives:**
- Understand object detection vs classification
- Detect and localize multiple objects in images
- Use DETR (DEtection TRansformer) models
- Visualize bounding boxes and labels

## Prerequisites

### Hardware Requirements

| Model Option | Model Name | Size | Min RAM | Recommended Setup | Notes |
|--------------|------------|------|---------|-------------------|-------|
| **CPU (Small)** | facebook/detr-resnet-50 | 159MB | 8GB | 8GB RAM, CPU | Manageable on CPU |
| **GPU (Medium)** | facebook/detr-resnet-101 | 232MB | 8GB | 10GB VRAM (RTX 4080) | Better accuracy |

### Software Requirements
- Python 3.8+
- Libraries: `transformers`, `torch`, `PIL`
- See `requirements.txt` for full list

## Overview

**Object Detection** finds and localizes multiple objects in an image.

**Difference from Classification:**
- Classification: "What is in the image?" → "cat"
- Detection: "What and where?" → "cat at (50, 100, 200, 300)"

**Use Cases:**
- Autonomous driving
- Security surveillance
- Retail analytics
- Sports analysis

**DETR (DEtection TRansformer):**
- End-to-end transformer for object detection
- No need for anchor boxes or NMS
- Simpler than traditional methods

## Expected Behaviors

### First Time Running
- **Model Download**: ~159MB for DETR-ResNet-50 (~2-3 minutes)
- Smaller than classification models due to efficient architecture
- Cached for future use

### Setup Cell Output
```
PyTorch version: 2.x.x
CUDA available: True/False
```

### Model Loading
```
Loading facebook/detr-resnet-50...
Model loaded successfully!
```
- **CPU**: 5-8 seconds
- **GPU**: 3-5 seconds

### Detection Output Format
```python
[
  {
    'label': 'person',
    'score': 0.9987,
    'box': {'xmin': 50, 'ymin': 100, 'xmax': 200, 'ymax': 300}
  },
  ...
]
```

### Bounding Box Format
- **xmin, ymin**: Top-left corner coordinates
- **xmax, ymax**: Bottom-right corner coordinates
- Coordinates in pixels relative to original image size

### Performance Expectations
- **Detection time**:
  - CPU: 5-10 seconds per image
  - GPU: 1-2 seconds per image
- **Slower than classification** due to finding multiple objects and locations

### Confidence Thresholds
- **threshold=0.9**: Only very confident detections (may miss some objects)
- **threshold=0.7**: Balanced (recommended default)
- **threshold=0.5**: More detections, but some may be incorrect
- Lower thresholds = more false positives

### Expected Detections
- Detects **COCO dataset classes** (80 categories):
  - People, vehicles, animals
  - Furniture, electronics
  - Food items, sports equipment
- Full list: [COCO Categories](https://cocodataset.org/#explore)

### Detection Quality
- **Single, clear object**: Detects with 90%+ confidence
- **Multiple objects**: Detects most, may miss small/occluded ones
- **Crowded scenes**: May merge nearby objects or miss some
- **Small objects**: Lower detection rate (<32x32 pixels)

### Common Observations
- Can detect **multiple objects** of same class (e.g., 3 people)
- Bounding boxes may not be pixel-perfect
- Occasionally detects background as object (false positive)
- Works best with well-lit, unoccluded objects

## Setup and Installation

In [ ]:
# Import required libraries
import torch
from transformers import AutoImageProcessor, AutoModelForObjectDetection, pipeline, set_seed
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
set_seed(1103)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Model Selection

In [ ]:
# CHOOSE YOUR MODEL:

# Option 1: CPU-friendly (recommended for beginners)
MODEL_NAME = "facebook/detr-resnet-50"  # 159MB

# Option 2: GPU-optimized (uncomment if you have RTX 4080 or similar)
# MODEL_NAME = "facebook/detr-resnet-101"  # 232MB, better accuracy

print(f"Selected model: {MODEL_NAME}")

## Helper Functions

In [ ]:
def load_image_from_url(url):
    """Load an image from a URL."""
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

def draw_bounding_boxes(image, detections, threshold=0.9):
    """
    Draw bounding boxes on an image.
    """
    draw = ImageDraw.Draw(image)
    
    for detection in detections:
        if detection['score'] < threshold:
            continue
        
        box = detection['box']
        label = detection['label']
        score = detection['score']
        
        # Draw rectangle
        x1, y1, x2, y2 = box['xmin'], box['ymin'], box['xmax'], box['ymax']
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
        
        # Draw label
        text = f"{label}: {score:.2f}"
        draw.text((x1, y1-15), text, fill="red")
    
    return image

## Method 1: Using Pipeline (Simplest)

In [ ]:
# Create object detection pipeline
print(f"Loading {MODEL_NAME}...")
detector = pipeline(
    "object-detection",
    model=MODEL_NAME,
    device=0 if torch.cuda.is_available() else -1
)

### Basic Object Detection

In [ ]:
# Load and detect objects
image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/coco_sample.png"
image = load_image_from_url(image_url)

print(f"Image size: {image.size}")

# Detect objects
detections = detector(image, threshold=0.9)

print(f"\n=== DETECTED {len(detections)} OBJECTS ===")
for i, detection in enumerate(detections, 1):
    print(f"{i}. {detection['label']:15s} - Score: {detection['score']:.4f}")
    print(f"   Box: {detection['box']}")

# Visualize
image_with_boxes = draw_bounding_boxes(image.copy(), detections, threshold=0.9)
print("\nImage with bounding boxes drawn (display in Jupyter)")
image_with_boxes  # Will display in Jupyter

### Adjusting Confidence Threshold

In [ ]:
# Compare different confidence thresholds
thresholds = [0.5, 0.7, 0.9]

for threshold in thresholds:
    detections = detector(image, threshold=threshold)
    print(f"\nThreshold {threshold}: {len(detections)} objects detected")
    for det in detections[:5]:  # Show top 5
        print(f"  - {det['label']:15s} ({det['score']:.3f})")

## Method 2: Using Model and Processor Directly

In [ ]:
# Load processor and model
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = AutoModelForObjectDetection.from_pretrained(MODEL_NAME)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"Model loaded on: {device}")

In [ ]:
# Detect with more control
image = load_image_from_url("https://images.unsplash.com/photo-1444212477490-ca407925329e?w=600")  # street scene

# Process image
inputs = processor(images=image, return_tensors="pt").to(device)

# Run detection
with torch.no_grad():
    outputs = model(**inputs)

# Post-process
target_sizes = torch.tensor([image.size[::-1]]).to(device)  # (height, width)
results = processor.post_process_object_detection(
    outputs, 
    target_sizes=target_sizes, 
    threshold=0.7
)[0]

print(f"\n=== DETECTED OBJECTS ===")
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    label_name = model.config.id2label[label.item()]
    print(f"{label_name:15s} - {score.item():.4f}")
    print(f"  Box: {box.tolist()}")

In [ ]:
# Model statistics
print("\n=== YOLOV8 MODEL FAMILY ===")

models_info = {
    'YOLOv8n': {'size': '6MB', 'params': '3.2M', 'mAP': '37.3', 'speed': '0.99ms'},
    'YOLOv8s': {'size': '22MB', 'params': '11.2M', 'mAP': '44.9', 'speed': '1.20ms'},
    'YOLOv8m': {'size': '52MB', 'params': '25.9M', 'mAP': '50.2', 'speed': '1.83ms'},
    'YOLOv8l': {'size': '87MB', 'params': '43.7M', 'mAP': '52.9', 'speed': '2.39ms'},
    'YOLOv8x': {'size': '218MB', 'params': '68.2M', 'mAP': '53.9', 'speed': '3.53ms'},
}

print(f"{'Model':<10} {'Size':<10} {'Params':<10} {'mAP@50-95':<12} {'Speed (GPU)'}")
print("-"*60)
for name, info in models_info.items():
    print(f"{name:<10} {info['size']:<10} {info['params']:<10} {info['mAP']:<12} {info['speed']}")

print("\n📊 Choose based on your hardware and speed requirements")
print("   - Nano (n): Edge devices, real-time video on CPU")
print("   - Small (s): Balanced option, good for most use cases")
print("   - Medium (m): Higher accuracy, still fast")
print("   - Large (l): Production systems with GPU")
print("   - XLarge (x): Maximum accuracy, requires powerful GPU")

In [ ]:
# Advanced: Object tracking in video
print("\n=== VIDEO OBJECT TRACKING ===")

# Process video (if you have a video file)
# results = yolo_model.track(
#     source="path/to/video.mp4",
#     show=True,  # Display results
#     save=True,  # Save output video
#     tracker="bytetrack.yaml"  # Tracking algorithm
# )

print("To process video:")
print("  results = yolo_model.track(source='video.mp4', show=True)")
print("\nYOLO can track objects across frames!")
print("Useful for:")
print("  - People counting")
print("  - Vehicle tracking")
print("  - Sports analytics")
print("  - Surveillance systems")

In [ ]:
# Performance comparison: DETR vs YOLO
import time

print("\n=== PERFORMANCE COMPARISON ===")

# Measure YOLO inference time
img = test_image_yolo

start = time.perf_counter()
for _ in range(10):
    _ = yolo_model(img, verbose=False)
yolo_time = (time.perf_counter() - start) / 10

print(f"YOLO average inference time: {yolo_time*1000:.2f} ms")
print(f"YOLO FPS: {1/yolo_time:.1f}")

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(f"{'Aspect':<20} {'DETR':<25} {'YOLOv8'}")
print("-"*70)
print(f"{'Model Size':<20} {'159MB (ResNet-50)':<25} {'6-218MB (n-x)'}")
print(f"{'Speed (GPU)':<20} {'~50ms':<25} {f'~{yolo_time*1000:.0f}ms'}")
print(f"{'FPS (GPU)':<20} {'~20':<25} {f'~{1/yolo_time:.0f}'}")
print(f"{'Accuracy (COCO)':<20} {'42.0 mAP':<25} {'45-53 mAP'}")
print(f"{'Best For':<20} {'Research':<25} {'Production'}")
print(f"{'Training':<20} {'Complex':<25} {'Easy'}")
print("="*70)

print("\n💡 **When to use YOLO:**")
print("  - Real-time applications (video, webcam)")
print("  - Production deployments")
print("  - Edge devices (mobile, embedded)")
print("  - Need for speed over research features")

print("\n💡 **When to use DETR:**")
print("  - Research projects")
print("  - Transformer-based architectures")
print("  - When speed is not critical")

In [ ]:
# Visualize detections
import matplotlib.pyplot as plt

# YOLO provides built-in visualization
annotated_img = results[0].plot()  # Returns annotated image as numpy array

# Display
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('YOLOv8 Object Detection', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Run detection on test image
# Use the same image as before for comparison
test_image_yolo = load_image_from_url(image_url)

# Run YOLO detection
results = yolo_model(test_image_yolo)

# Get detections
detections_yolo = results[0]

print("="*70)
print("YOLO DETECTION RESULTS")
print("="*70)

# Print detections
for i, box in enumerate(detections_yolo.boxes):
    class_id = int(box.cls[0])
    confidence = float(box.conf[0])
    bbox = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
    
    class_name = yolo_model.names[class_id]
    
    print(f"{i+1}. {class_name}")
    print(f"   Confidence: {confidence:.4f}")
    print(f"   Bounding box: [{bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f}]")
    print()

print(f"Total detections: {len(detections_yolo.boxes)}")
print("="*70)

In [ ]:
# Install YOLOv8 (if not already installed)
# !pip install ultralytics

from ultralytics import YOLO
import cv2
from PIL import Image
import numpy as np

# Load YOLOv8 model
print("Loading YOLOv8...")

# Choose model size:
# yolov8n.pt = Nano (6MB, fastest)
# yolov8s.pt = Small (22MB, fast)
# yolov8m.pt = Medium (52MB, balanced)
# yolov8l.pt = Large (87MB, accurate)
# yolov8x.pt = XLarge (218MB, most accurate)

MODEL_SIZE = "yolov8s.pt"  # Change to n/s/m/l/x
yolo_model = YOLO(MODEL_SIZE)

print(f"✓ YOLOv8-{MODEL_SIZE[6]} loaded")
print(f"Model size: {MODEL_SIZE}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## Method 3: Real-Time Detection with YOLOv8

**YOLOv8** (You Only Look Once) is the state-of-the-art real-time object detector:
- **Industry standard** - Most widely used in production
- **Blazing fast** - 100+ FPS on GPU
- **Multiple sizes** - Nano (6MB) to XLarge (218MB)
- **Better accuracy** - Than DETR on most benchmarks
- **Easy to use** - Simple API from Ultralytics

**When to use**: Production applications, real-time video, deployment on edge devices.

## Practical Applications

### Example 1: Count Objects by Category

In [ ]:
from collections import Counter

def count_objects(image, threshold=0.7):
    """Count detected objects by category."""
    detections = detector(image, threshold=threshold)
    labels = [d['label'] for d in detections]
    counts = Counter(labels)
    
    print(f"\n=== OBJECT COUNTS (threshold={threshold}) ===")
    for label, count in counts.most_common():
        print(f"{label:15s}: {count}")
    
    return counts

# Test
test_image = load_image_from_url("https://images.unsplash.com/photo-1449965408869-eaa3f722e40d?w=600")  # food
counts = count_objects(test_image, threshold=0.6)

### Example 2: Filter by Object Type

In [ ]:
def find_specific_objects(image, target_objects=['person', 'car', 'dog'], threshold=0.7):
    """
    Find specific types of objects in an image.
    """
    detections = detector(image, threshold=threshold)
    
    found = {obj: [] for obj in target_objects}
    
    for detection in detections:
        label = detection['label']
        if label in target_objects:
            found[label].append(detection)
    
    print("\n=== SPECIFIC OBJECT SEARCH ===")
    for obj, instances in found.items():
        print(f"\n{obj.upper()}: {len(instances)} found")
        for inst in instances:
            print(f"  Score: {inst['score']:.3f}, Box: {inst['box']}")
    
    return found

# Test
test_image = load_image_from_url(image_url)
found_objects = find_specific_objects(test_image, target_objects=['person', 'car', 'cup'])

### Example 3: Multi-Image Analysis

In [ ]:
# Analyze multiple images
image_urls = [
    "https://images.unsplash.com/photo-1606041011872-596597976b25?w=400",  # street
    "https://images.unsplash.com/photo-1504674900247-0877df9cc836?w=400",  # food
]

for i, url in enumerate(image_urls, 1):
    print(f"\n{'='*50}")
    print(f"IMAGE {i}")
    print('='*50)
    
    try:
        img = load_image_from_url(url)
        detections = detector(img, threshold=0.8)
        
        print(f"Found {len(detections)} objects:")
        for det in detections:
            print(f"  - {det['label']:15s} ({det['score']:.3f})")
    except Exception as e:
        print(f"Error: {e}")

## Performance Benchmarking

## State-of-the-Art Open Models (Not Covered)

While we've covered DETR and YOLOv8, there are several groundbreaking detection models that push the boundaries of what's possible. These include self-supervised models, open-vocabulary detectors, and universal segmentation systems.

### Top SOTA Object Detection Models

#### 1. 🦕 DINO (Facebook/Meta)
**Self-supervised vision transformer for detection**
- **Why it's special**: Self-supervised pre-training, no labeled data needed for initial training
- **Performance**: 63.3 AP on COCO, excellent feature representations
- **Model Card**: [facebook/dinov2-large](https://huggingface.co/facebook/dinov2-large)
- **Paper**: [DINO: Emerging Properties in Self-Supervised Vision Transformers](https://arxiv.org/abs/2104.14294)
- **Size**: 1.1GB (ViT-Large backbone)

#### 2. 🔍 Grounding DINO
**Open-vocabulary object detection with text prompts**
- **Why it's special**: Detect ANY object described in text, zero-shot detection
- **Performance**: Combines DINO + text understanding, detects novel categories
- **Model Card**: [IDEA-Research/grounding-dino-base](https://huggingface.co/IDEA-Research/grounding-dino-base)
- **Paper**: [Grounding DINO: Marrying DINO with Grounded Pre-Training](https://arxiv.org/abs/2303.05499)
- **Size**: 680MB

#### 3. 🎯 SAM (Segment Anything Model - Meta)
**Universal image segmentation at pixel level**
- **Why it's special**: Segments ANY object with prompts (points, boxes, text), 1B+ mask dataset
- **Performance**: Zero-shot segmentation, works on any domain
- **Model Card**: [facebook/sam-vit-huge](https://huggingface.co/facebook/sam-vit-huge)
- **Paper**: [Segment Anything](https://arxiv.org/abs/2304.02643)
- **Size**: 2.4GB (ViT-Huge), also 375MB (ViT-Base) available

#### 4. 📝 DETA (Detection with Text Assistance)
**Object detection enhanced with text prompts**
- **Why it's special**: Uses text descriptions to improve detection accuracy
- **Performance**: 63.5 AP on COCO, better than DETR with similar size
- **Model Card**: Available on GitHub (IDEA-Research/DETA)
- **Paper**: [NMS Strikes Back](https://arxiv.org/abs/2212.06137)
- **Size**: ~300MB

### Why Not Covered?

These advanced models require:
- **GPU Memory**: 16-48GB VRAM for SAM-huge, 12GB+ for others
- **Inference Time**: 5-30 seconds per image (vs 1-2s for DETR/YOLO)
- **Complex Setup**: Some require additional dependencies (CLIP, specialized processors)
- **Specialized Use Cases**: Most useful for specific applications rather than general learning

DETR and YOLO provide solid foundations before exploring these advanced techniques!

### Learning Path Recommendation

1. **Start here**: Master DETR and YOLOv8 (this notebook)
2. **Next step**: Try SAM for segmentation tasks (use ViT-Base version)
3. **Open-vocabulary**: Experiment with Grounding DINO for flexible detection
4. **Research**: Explore DINO for self-supervised learning projects

### Benchmarks & Leaderboards

- **COCO Object Detection mAP** (val2017):
  - DETR-ResNet-50: 42.0 mAP
  - YOLOv8-large: 52.9 mAP
  - DINO (ViT-Large): 63.3 mAP
  - DETA: 63.5 mAP
  - Grounding DINO: ~57 mAP (zero-shot capable)

- **SAM** (Segmentation):
  - 91.6 mIoU on SA-V dataset
  - Works on 11M+ images, 1.1B+ masks

- **Explore rankings**: [Papers With Code - Object Detection on COCO](https://paperswithcode.com/sota/object-detection-on-coco)

### Quick Comparison Table

| Model | Size | Speed | mAP | Best For |
|-------|------|-------|-----|----------|
| **DETR** ⭐ | 159MB | Medium | 42.0 | Learning transformers |
| **YOLOv8** ⭐ | 6-218MB | Very Fast | 37-54 | Real-time, production |
| **DINO** | 1.1GB | Slow | 63.3 | High accuracy, research |
| **Grounding DINO** | 680MB | Slow | 57.0 | Open-vocabulary detection |
| **SAM** | 375MB-2.4GB | Very Slow | N/A | Universal segmentation |
| **DETA** | 300MB | Medium | 63.5 | Text-assisted detection |

### Special Capabilities

**🌟 What makes these models unique:**

- **SAM**: Click on any object to segment it, no training needed
- **Grounding DINO**: Detect "a red car with scratches" without training on that specific class
- **DINO**: Learn powerful features without any labels, excellent for transfer learning
- **DETA**: Improve detection by describing what you're looking for

**💡 Tip**: For production, stick with YOLOv8. For research or specialized applications (zero-shot detection, segmentation), explore SAM and Grounding DINO!

## Exercises

1. **Custom Images**: Test with your own images. What objects are detected?

2. **Threshold Tuning**: Find the optimal threshold for different image types.

3. **Object Counter**: Build a function that counts total objects across multiple images.

4. **Model Comparison**: Compare DETR-ResNet-50 vs DETR-ResNet-101 (if you have GPU).

5. **Visualization**: Enhance the bounding box drawing with different colors per class.

In [ ]:
# Your code here for exercises


## Key Takeaways

✅ **Object detection** finds both what and where objects are

✅ **DETR** is a modern transformer-based detector

✅ **Confidence threshold** controls detection sensitivity

✅ **Bounding boxes** are in (xmin, ymin, xmax, ymax) format

✅ Detection is more computationally intensive than classification

## Next Steps

- Try **Notebook 06**: OCR for text extraction from images
- Explore other detection models on [HuggingFace Hub](https://huggingface.co/models?pipeline_tag=object-detection)
- Learn about instance segmentation for pixel-level detection

## Resources

- [DETR Paper](https://arxiv.org/abs/2005.12872)
- [Object Detection Guide](https://huggingface.co/docs/transformers/tasks/object_detection)
- [COCO Dataset](https://cocodataset.org/)